In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import libraries
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import os
import numpy as np
import pandas as pd
import math, random
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
sns.set_style('whitegrid')
from scipy import stats
import missingno as msno

# display all columns
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_row', None)

In [ ]:
# load data
train_data = pd.read_csv('/content/drive/MyDrive/최종프로젝트/physionet2012_train.csv')
valid_data = pd.read_csv('/content/drive/MyDrive/최종프로젝트/physionet2012_valid.csv')
test_data = pd.read_csv('/content/drive/MyDrive/최종프로젝트/physionet2012_test.csv')

train_data.drop(['Unnamed: 0'], axis=1, inplace=True)
valid_data.drop(['Unnamed: 0'], axis=1, inplace=True)
test_data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
# labeling
# train_data
train_data['label'] = 0
train_data.loc[(2 <= train_data['survival'])&(train_data['survival'] <= train_data['LOS']), 'label'] = 1
train_data.loc[train_data['survival'] > train_data['LOS'], 'label'] = 0
train_data.loc[train_data['survival'] == -1, 'label'] = 0

# valid_data
valid_data['label'] = 0
valid_data.loc[(2 <= valid_data['survival'])&(valid_data['survival'] <= valid_data['LOS']), 'label'] = 1
valid_data.loc[valid_data['survival'] > valid_data['LOS'], 'label'] = 0
valid_data.loc[valid_data['survival'] == -1, 'label'] = 0

# test_data
test_data['label'] = 0
test_data.loc[(2 <= test_data['survival'])&(test_data['survival'] <= test_data['LOS']), 'label'] = 1
test_data.loc[test_data['survival'] > test_data['LOS'], 'label'] = 0
test_data.loc[test_data['survival'] == -1, 'label'] = 0

In [ ]:
train_df = train_data.copy()
valid_df = valid_data.copy()
test_df = test_data.copy()

In [ ]:
# ffill
train_df.groupby(['pid']).fillna(method='ffill', inplace=True)
valid_df.groupby(['pid']).fillna(method='ffill', inplace=True)
test_df.groupby(['pid']).fillna(method='ffill', inplace=True)

# median
train_df.fillna(train_df.median(), inplace=True)
valid_df.fillna(valid_df.median(), inplace=True)
test_df.fillna(test_df.median(), inplace=True)

In [ ]:
# Gender -1값 1로 바꾸기
train_df.loc[train_df['Gender']==-1, 'Gender'] = 1 

In [ ]:
# rec_time to sec
def to_secs(rec):
    dh, m, s = rec.split(':')
    if 'days' in dh:
        d, h = dh.split(' days, ')
    elif 'day' in dh:
        d, h = dh.split(' day, ')
    else:
        d = 0
        h = dh

    secs = int(d)*24*60*60 + int(h)*60*60 + int(m)*60 + int(s)
    return secs

def timetosecond(df):
    df['rec_time_sec'] = df['rec_time'].replace('0', '00:00:00')
    df['rec_time_sec'] = df['rec_time'].apply(to_secs)
    
    return df

In [ ]:
train_df = timetosecond(train_df)
valid_df = timetosecond(valid_df)
test_df = timetosecond(test_df)

In [ ]:
input_cols = [x for x in train_df.columns if x not in ['pid', 'Gender', 'ICUType', 'MechVent', 'rec_time', 'Tropl', 'TropT', 'male', 'female', 'survival', 'surv_los', 'LOS', 'label']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_df[input_cols])
scaled_train = scaler.transform(train_df[input_cols])
scaled_valid = scaler.transform(valid_df[input_cols])
scaled_test = scaler.transform(test_df[input_cols])
# scaled_train_df = pd.DataFrame(data=scaled_train, columns=ordinary_train.feature_names)

In [ ]:
train_df2 = train_df.copy()
valid_df2 = valid_df.copy()
test_df2 = test_df.copy()

In [ ]:
train_df2[input_cols] = scaled_train
valid_df2[input_cols] = scaled_valid
test_df2[input_cols] = scaled_test

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
train_df3 = []
for pid in tqdm(train_df2.pid.unique()):
    pid_df = train_df2.query('pid==@pid')
    for col in input_cols:
        delta_col = f'{col}_delta'
        dt_values = np.array([0]+list(pid_df[col].values[1:] - pid_df[col].values[:-1]))
        pid_df[delta_col] = dt_values
    train_df3.append(pid_df)

  0%|          | 0/3240 [00:00<?, ?it/s]

In [ ]:
valid_df3 = []
for pid in tqdm(valid_df2.pid.unique()):
    pid_df = valid_df2.query('pid==@pid')
    for col in input_cols:
        delta_col = f'{col}_delta'
        dt_values = np.array([0]+list(pid_df[col].values[1:] - pid_df[col].values[:-1]))
        pid_df[delta_col] = dt_values
    valid_df3.append(pid_df)

  0%|          | 0/360 [00:00<?, ?it/s]

In [ ]:
test_df3 = []
for pid in tqdm(test_df2.pid.unique()):
    pid_df = test_df2.query('pid==@pid')
    for col in input_cols:
        delta_col = f'{col}_delta'
        dt_values = np.array([0]+list(pid_df[col].values[1:] - pid_df[col].values[:-1]))
        pid_df[delta_col] = dt_values
    test_df3.append(pid_df)

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
train_df3 = pd.concat(train_df3).reset_index(drop=True)
valid_df3 = pd.concat(valid_df3).reset_index(drop=True)
test_df3 = pd.concat(test_df3).reset_index(drop=True)

In [ ]:
train_df3

,pid,rec_time,age,Gender,ICUType,ALP,ALT,AST,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,FiO2,GCS,Glucose,HCO3,HCT,HR,K,Lactate,Mg,Na,PaCO2,PaO2,Platelets,RR,SPO2,BT,Tropl,TroponinI,TropT,TroponinT,Urine,WBC,pH,MechVent,survival,SBP,DBP,MBP,male,female,surv_los,LOS,label,rec_time_sec,age_delta,ALP_delta,ALT_delta,AST_delta,Albumin_delta,BUN_delta,Bilirubin_delta,Cholesterol_delta,Creatinine_delta,FiO2_delta,GCS_delta,Glucose_delta,HCO3_delta,HCT_delta,HR_delta,K_delta,Lactate_delta,Mg_delta,Na_delta,PaCO2_delta,PaO2_delta,Platelets_delta,RR_delta,SPO2_delta,BT_delta,TroponinI_delta,TroponinT_delta,Urine_delta,WBC_delta,pH_delta,SBP_delta,DBP_delta,MBP_delta,rec_time_sec_delta
0,142219,0:00:00,0.493333,0.0,3.0,0.03192,0.003662,0.003148,0.475,0.101523,0.016807,0.417219,0.040909,0.367089,0.833333,0.103266,0.400000,0.388350,0.286667,0.109005,0.062069,0.197183,0.518987,0.388164,0.244,0.160423,0.310345,0.959459,0.916528,NaN,0.05317,NaN,0.008434,0.006364,0.060832,0.008692,1.0,-1,0.393220,0.216418,0.256667,1,0,-8,7,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,142219,0:44:00,0.493333,0.0,3.0,0.03192,0.003662,0.003148,0.475,0.101523,0.016807,0.417219,0.040909,0.746835,0.833333,0.103266,0.400000,0.388350,0.343333,0.109005,0.062069,0.197183,0.518987,0.388164,0.244,0.160423,0.310345,0.959459,0.916528,NaN,0.05317,NaN,0.008434,0.045455,0.060832,0.008692,1.0,-1,0.393220,0.216418,0.256667,1,0,-8,7,0,0.015278,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.379747,0.000000,0.000000,0.000000,0.000000,0.056667,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.039091,0.000000,0.0,0.000000,0.000000,0.000000,0.015278
2,142219,1:14:00,0.493333,0.0,3.0,0.03192,0.003662,0.003148,0.475,0.101523,0.016807,0.417219,0.040909,0.367089,0.833333,0.103266,0.400000,0.388350,0.333333,0.109005,0.062069,0.197183,0.518987,0.388164,0.244,0.160423,0.310345,0.959459,0.916528,NaN,0.05317,NaN,0.008434,0.006364,0.060832,0.008692,1.0,-1,0.000000,0.000000,0.746667,1,0,-8,7,0,0.025694,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,-0.379747,0.000000,0.000000,0.000000,0.000000,-0.010000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,-0.039091,0.000000,0.0,-0.393220,-0.216418,0.490000,0.010417
3,142219,1:44:00,0.493333,0.0,3.0,0.03192,0.003662,0.003148,0.475,0.101523,0.016807,0.417219,0.040909,0.367089,0.833333,0.103266,0.400000,0.388350,0.353333,0.109005,0.062069,0.197183,0.518987,0.388164,0.244,0.160423,0.310345,0.959459,0.916528,NaN,0.05317,NaN,0.008434,0.006364,0.060832,0.008692,1.0,-1,0.000000,0.000000,0.450000,1,0,-8,7,0,0.036111,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,-0.296667,0.010417
4,142219,1:54:00,0.493333,0.0,3.0,0.03192,0.003662,0.003148,0.475,0.101523,0.016807,0.417219,0.040909,0.367089,0.833333,0.103266,0.400000,0.388350,0.350000,0.109005,0.062069,0.197183,0.518987,0.388164,0.244,0.160423,0.310345,0.959459,0.916528,NaN,0.05317,NaN,0.008434,0.040909,0.060832,0.008692,1.0,-1,0.735593,0.402985,0.443333,1,0,-8,7,0,0.039583,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.003333,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.034545,0.000000,0.0,0.735593,0.402985,-0.006667,0.003472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242608,138581,"1 day, 20:36:00",0.506667,1.0,4.0,0.03192,0.003662,0.003148,0.475,0.101523,0.016807,0.41

In [ ]:
def get_padded_df(df):
    padded_df = []
    p = df['pid'].unique()
    max_len = 70
    for ep in tqdm(p):
        if len(df[df['pid']==ep]) >= max_len:
            ep_df = df[df['pid']==ep][:max_len]
            padded_df.append(ep_df)
        else:
            empty_df = pd.DataFrame(index=range(0,70), columns = df.columns) 
            empty_df[['pid']] = ep
            empty_df[['age']] = df[df['pid']==ep]['age'].values[0]
            empty_df[['Gender']] = df[df['pid']==ep]['Gender'].values[0]
            empty_df[['ICUType']] = df[df['pid']==ep]['ICUType'].values[0]
            empty_df[['label']] = df[df['pid']==ep]['label'].values[0]
            empty_df.iloc[-len(df[df['pid']==ep]):] = df[df['pid']==ep]
            empty_df.fillna(0, inplace=True)
            padded_df.append(empty_df)
    return pd.concat(padded_df).reset_index(drop=True)

In [ ]:
pad_train_df = get_padded_df(train_df3)
pad_valid_df = get_padded_df(valid_df3)
pad_test_df = get_padded_df(test_df3)

  0%|          | 0/3240 [00:00<?, ?it/s]

  0%|          | 0/360 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
# pad_train_df.to_csv('/content/drive/MyDrive/최종프로젝트/preprocessed_all_train_delta.csv')
# pad_valid_df.to_csv('/content/drive/MyDrive/최종프로젝트/preprocessed_all_valid_delta.csv')
# pad_test_df.to_csv('/content/drive/MyDrive/최종프로젝트/preprocessed_all_test_delta.csv')

# Model

In [ ]:
# load data
pad_train_df = pd.read_csv('/content/drive/MyDrive/최종프로젝트/preprocessed_all_train_delta.csv')
pad_valid_df = pd.read_csv('/content/drive/MyDrive/최종프로젝트/preprocessed_all_valid_delta.csv')
pad_test_df = pd.read_csv('/content/drive/MyDrive/최종프로젝트/preprocessed_all_test_delta.csv')

pad_train_df.drop(['Unnamed: 0'], axis=1, inplace=True)
pad_valid_df.drop(['Unnamed: 0'], axis=1, inplace=True)
pad_test_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
delta_columns = [x for x in pad_train_df.columns if x not in ['pid', 'rec_time', 'rec_time_sec' 'Gender', 'ICUType', 'MechVent', 'Tropl', 'TropT', 'male', 'female', 'survival', 'surv_los', 'LOS', 'label']]

In [ ]:
delta_vital_columns = ['age', 'GCS', 'HR', 'RR', 'SPO2', 'BT', 'SBP', 'DBP', 'MBP', 'GCS_delta', 'HR_delta', 'RR_delta', 'SPO2_delta', 'BT_delta', 'SBP_delta', 'DBP_delta', 'MBP_delta', 'rec_time_sec_delta']

In [ ]:
# splitting dataset(original +delta)
train_X = pad_train_df[delta_columns].values
train_y = pad_train_df['label'].values

valid_X = pad_valid_df[delta_columns].values
valid_y = pad_valid_df['label'].values

test_X = pad_test_df[delta_columns].values
test_y = pad_test_df['label'].values

# type
train_X = train_X.astype(float)
train_y = train_y.astype(int)

valid_X = valid_X.astype(float)
valid_y = valid_y.astype(int)

test_X = test_X.astype(float)
test_y = test_y.astype(int)

In [ ]:
# splitting dataset(original vital + delta)
train_X = pad_train_df[vital_delta_columns].values
train_y = pad_train_df['label'].values

valid_X = pad_valid_df[vital_delta_columns].values
valid_y = pad_valid_df['label'].values

test_X = pad_test_df[vital_delta_columns].values
test_y = pad_test_df['label'].values

# type
train_X = train_X.astype(float)
train_y = train_y.astype(int)

valid_X = valid_X.astype(float)
valid_y = valid_y.astype(int)

test_X = test_X.astype(float)
test_y = test_y.astype(int)

In [ ]:
# imbalanced-learn 패키지
from imblearn.over_sampling import *
from imblearn.over_sampling import SMOTE

# 검증 데이터나 테스트 데이터가 아닌 학습데이터에서만 오버샘플링 사용할 것
# smote = SMOTE(random_state=0)
train_X_over, train_y_over = SMOTE(random_state=0).fit_resample(train_X, train_y)

In [ ]:
# reshape
print(train_X_over.shape, train_y_over.shape)
print(valid_X.shape, valid_y.shape)
print(test_X.shape, test_y.shape)

max_len = 70

train_X_3d = train_X_over.reshape(-1, max_len, train_X_over.shape[-1])
train_y_3d = train_y_over[::70]
valid_X_3d = valid_X.reshape(-1, max_len, valid_X.shape[-1])
valid_y_3d = valid_y[::70]
test_X_3d = test_X.reshape(-1, max_len, test_X.shape[-1])
test_y_3d = test_y[::70]

print(train_X_3d.shape, train_y_3d.shape)
print(valid_X_3d.shape, valid_y_3d.shape)
print(test_X_3d.shape, test_y_3d.shape)

(393820, 18) (393820,)
(25200, 18) (25200,)
(28000, 18) (28000,)
(5626, 70, 18) (5626,)
(360, 70, 18) (360,)
(400, 70, 18) (400,)


In [ ]:
feature_dim = train_X_over.shape[-1]

In [ ]:
# setup
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import roc_auc_score

from keras import layers
from keras import Sequential, Model
# from keras.preprocessing.sequence import pad_sequences
from pprint import pprint

from keras.models import Sequential             
from keras.layers import Dense, Dropout, BatchNormalization, SimpleRNN, LSTM, Bidirectional, BatchNormalization, Activation, GRU, Flatten, Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping 
from keras.optimizers import RMSprop, Adam
from keras import models
from keras import layers
from keras.preprocessing import sequence
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical

from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Flatten, Dropout
from keras.layers import SimpleRNN, LSTM, GRU

In [ ]:
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(max_len, feature_dim)))
cnn_model.add(Dropout(0.2))
cnn_model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(8, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))
cnn_model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(curve="ROC", name='auroc')])

In [ ]:
# cnn2(maxpooling)
cnn2_model = Sequential()
cnn2_model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(max_len, feature_dim)))
cnn2_model.add(MaxPooling1D())
cnn2_model.add(Conv1D(64, kernel_size=3, activation='relu'))
cnn2_model.add(Flatten())
cnn2_model.add(Dense(16, activation='relu'))
cnn2_model.add(BatchNormalization())
cnn2_model.add(Dropout(0.2))
cnn2_model.add(Dense(1, activation='sigmoid'))
cnn2_model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(curve="ROC", name='auroc')])

In [ ]:
# gru
gru_model = Sequential()
gru_model.add(Bidirectional(GRU(128, input_shape = (max_len, feature_dim), return_sequences=True, activation='tanh')))
gru_model.add(Dropout(0.25)) 
gru_model.add(Bidirectional(GRU(64, return_sequences=True, activation="tanh"))) 
gru_model.add(Dropout(0.25)) 
gru_model.add(Bidirectional(GRU(32, return_sequences=False, activation="tanh"))) 
gru_model.add(Dropout(0.2)) 
gru_model.add(Dense(16, activation='tanh'))
gru_model.add(Dense(8, activation='tanh'))
gru_model.add(Dense(1, activation='sigmoid'))
gru_model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(curve="ROC", name='auroc')])

In [ ]:
# gru2
gru2_model = Sequential()
gru2_model.add(GRU(128, input_shape = (max_len, feature_dim), return_sequences=True, activation='tanh'))
gru2_model.add(Dropout(0.5)) 
gru2_model.add(GRU(64, return_sequences=True, activation="tanh"))
gru2_model.add(Dropout(0.25)) 
gru2_model.add(GRU(32, return_sequences=True, activation="tanh"))
gru2_model.add(Dropout(0.25)) 
gru2_model.add(GRU(16, return_sequences=False, activation="tanh")) 
gru2_model.add(Dropout(0.2)) 
gru2_model.add(Dense(8, activation='tanh'))
gru2_model.add(Dense(1, activation='sigmoid'))
gru2_model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(curve="ROC", name='auroc')])

In [ ]:
cnn_model_history = cnn_model.fit(train_X_3d, train_y_3d, epochs=100, batch_size=50, validation_data=(valid_X_3d, valid_y_3d), verbose=1)
cnn_history = cnn_model_history.history

Epoch 1/100
113/113 [==============================] - 4s 29ms/step - loss: 0.6978 - auroc: 0.5006 - val_loss: 0.7043 - val_auroc: 0.4939
Epoch 2/100
113/113 [==============================] - 3s 25ms/step - loss: 0.6973 - auroc: 0.4963 - val_loss: 0.7082 - val_auroc: 0.5108
Epoch 3/100
113/113 [==============================] - 3s 25ms/step - loss: 0.6932 - auroc: 0.5189 - val_loss: 0.7129 - val_auroc: 0.5177
Epoch 4/100
113/113 [==============================] - 3s 27ms/step - loss: 0.6926 - auroc: 0.5294 - val_loss: 0.7117 - val_auroc: 0.5051
Epoch 5/100
113/113 [==============================] - 5s 40ms/step - loss: 0.6921 - auroc: 0.5252 - val_loss: 0.7177 - val_auroc: 0.5340
Epoch 6/100
113/113 [==============================] - 3s 26ms/step - loss: 0.6909 - auroc: 0.5414 - val_loss: 0.7160 - val_auroc: 0.5093
Epoch 7/100
113/113 [==============================] - 3s 26ms/step - loss: 0.6879 - auroc: 0.5563 - val_loss: 0.7120 - val_auroc: 0.5204
Epoch 8/100
113/113 [=============

In [ ]:
cnn2_model_history = cnn2_model.fit(train_X_3d, train_y_3d, epochs=100, batch_size=50, validation_data=(valid_X_3d, valid_y_3d), verbose=1)
cnn2_history = cnn2_model_history.history

Epoch 1/100
113/113 [==============================] - 6s 38ms/step - loss: 0.6186 - auroc: 0.7067 - val_loss: 0.6981 - val_auroc: 0.5450
Epoch 2/100
113/113 [==============================] - 3s 23ms/step - loss: 0.5013 - auroc: 0.8522 - val_loss: 0.6888 - val_auroc: 0.5605
Epoch 3/100
113/113 [==============================] - 3s 23ms/step - loss: 0.4384 - auroc: 0.8966 - val_loss: 0.6525 - val_auroc: 0.5681
Epoch 4/100
113/113 [==============================] - 3s 23ms/step - loss: 0.3942 - auroc: 0.9168 - val_loss: 0.5780 - val_auroc: 0.5735
Epoch 5/100
113/113 [==============================] - 3s 22ms/step - loss: 0.3582 - auroc: 0.9300 - val_loss: 0.5329 - val_auroc: 0.5781
Epoch 6/100
113/113 [==============================] - 3s 24ms/step - loss: 0.3367 - auroc: 0.9352 - val_loss: 0.4555 - val_auroc: 0.5740
Epoch 7/100
113/113 [==============================] - 3s 23ms/step - loss: 0.3151 - auroc: 0.9406 - val_loss: 0.4701 - val_auroc: 0.5807
Epoch 8/100
113/113 [=============

In [ ]:
gru_model_history = gru_model.fit(train_X_3d, train_y_3d, epochs=100, batch_size=50, validation_data=(valid_X_3d, valid_y_3d), verbose=1)
gru_history = gru_model_history.history 

Epoch 1/100
113/113 [==============================] - 67s 465ms/step - loss: 0.6779 - auroc: 0.6281 - val_loss: 0.7089 - val_auroc: 0.5035
Epoch 2/100
113/113 [==============================] - 53s 469ms/step - loss: 0.6523 - auroc: 0.6984 - val_loss: 0.6898 - val_auroc: 0.5476
Epoch 3/100
113/113 [==============================] - 50s 443ms/step - loss: 0.6368 - auroc: 0.7091 - val_loss: 0.6966 - val_auroc: 0.5471
Epoch 4/100
113/113 [==============================] - 49s 437ms/step - loss: 0.6277 - auroc: 0.7042 - val_loss: 0.6861 - val_auroc: 0.5549
Epoch 5/100
113/113 [==============================] - 49s 436ms/step - loss: 0.6187 - auroc: 0.7077 - val_loss: 0.6955 - val_auroc: 0.5578
Epoch 6/100
113/113 [==============================] - 52s 460ms/step - loss: 0.6078 - auroc: 0.7298 - val_loss: 0.6826 - val_auroc: 0.5572
Epoch 7/100
113/113 [==============================] - 49s 437ms/step - loss: 0.6078 - auroc: 0.7104 - val_loss: 0.6709 - val_auroc: 0.5604
Epoch 8/100
113/113 

In [ ]:
gru2_model_history = gru2_model.fit(train_X_3d, train_y_3d, epochs=100, batch_size=50, validation_data=(valid_X_3d, valid_y_3d), verbose=1)
gru2_history = gru2_model_history.history

Epoch 1/100
113/113 [==============================] - 40s 281ms/step - loss: 0.6938 - auroc: 0.5046 - val_loss: 0.7062 - val_auroc: 0.5183
Epoch 2/100
113/113 [==============================] - 28s 250ms/step - loss: 0.6929 - auroc: 0.5169 - val_loss: 0.7002 - val_auroc: 0.6095
Epoch 3/100
113/113 [==============================] - 28s 248ms/step - loss: 0.6912 - auroc: 0.5474 - val_loss: 0.6961 - val_auroc: 0.5919
Epoch 4/100
113/113 [==============================] - 29s 254ms/step - loss: 0.6904 - auroc: 0.5645 - val_loss: 0.6910 - val_auroc: 0.6226
Epoch 5/100
113/113 [==============================] - 28s 248ms/step - loss: 0.6894 - auroc: 0.5806 - val_loss: 0.6848 - val_auroc: 0.6302
Epoch 6/100
113/113 [==============================] - 28s 247ms/step - loss: 0.6878 - auroc: 0.6078 - val_loss: 0.6812 - val_auroc: 0.6358
Epoch 7/100
113/113 [==============================] - 28s 251ms/step - loss: 0.6870 - auroc: 0.6217 - val_loss: 0.6786 - val_auroc: 0.6397
Epoch 8/100
113/113 

In [ ]:
# predict
cnn_pred = cnn_model.predict(test_X_3d)
cnn2_pred = cnn2_model.predict(test_X_3d)
gru_pred = gru_model.predict(test_X_3d)
gru2_pred = gru2_model.predict(test_X_3d)

13/13 [==============================] - 2s 41ms/step


In [ ]:
# score(original+delta_dynamic)
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_curve, classification_report, accuracy_score, roc_auc_score, average_precision_score
print('CNN_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn_pred))
print('CNN2_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn2_pred))
print('GRU_ROC AUC Score: ', roc_auc_score(test_y_3d, gru_pred))
print('GRU2_ROC AUC Score: ', roc_auc_score(test_y_3d, gru2_pred))

CNN_ROC AUC Score:  0.7734305923961096
CNN2_ROC AUC Score:  0.793158709106985
GRU_ROC AUC Score:  0.7500552608311228
GRU2_ROC AUC Score:  0.688605216622458


In [ ]:
# score(original vital + delta_dynamic)
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_curve, classification_report, accuracy_score, roc_auc_score, average_precision_score
print('CNN_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn_pred))
print('CNN2_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn2_pred))
print('GRU_ROC AUC Score: ', roc_auc_score(test_y_3d, gru_pred))
print('GRU2_ROC AUC Score: ', roc_auc_score(test_y_3d, gru2_pred))

CNN_ROC AUC Score:  0.743866047745358
CNN2_ROC AUC Score:  0.7282272325375773
GRU_ROC AUC Score:  0.7102674624226348
GRU2_ROC AUC Score:  0.7157935455349248


In [ ]:
import os

model_dir = '/content/drive/MyDrive/최종프로젝트/model5/'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

modelpath='/content/drive/MyDrive/최종프로젝트/model5/'

In [ ]:
cnn_model.save(modelpath+'cnn_model_delta.h5')
cnn2_model.save(modelpath+'cnn2_model_delta.h5')
gru_model.save(modelpath+'gru_model_delta.h5')
gru2_model.save(modelpath+'gru2_model_delta.h5')

In [ ]:
cnn_model.save(modelpath+'cnn_model_vital_delta.h5')
cnn2_model.save(modelpath+'cnn2_model__vital_delta.h5')
gru_model.save(modelpath+'gru_model_vital_delta.h5')
gru2_model.save(modelpath+'gru2_model_vital_delta.h5')